In [ ]:
pip install -q kaggle

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download thomasdubail/brain-tumors-256x256

In [ ]:
! unzip brain-tumors-256x256.zip

In [ ]:
import os
import shutil
import random

# Set seed for reproducibility
random.seed(42)

# Paths
source_dir = '/content/Data'
base_dir = '/content/output'  # Output folder
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# Create output directories
for split_dir in [train_dir, test_dir]:
    for category in ['glioma_tumor', 'meningioma_tumor', 'normal', 'pituitary_tumor']:
        os.makedirs(os.path.join(split_dir, category), exist_ok=True)

# Split function
def split_data(source, train_dest, test_dest, split_ratio=0.8):
    files = [f for f in os.listdir(source) if f.endswith('.jpg')]
    random.shuffle(files)

    split_point = int(len(files) * split_ratio)
    train_files = files[:split_point]
    test_files = files[split_point:]

    for file_list, dest_dir in [(train_files, train_dest), (test_files, test_dest)]:
        for fname in file_list:
            src_path = os.path.join(source, fname)
            dst_path = os.path.join(dest_dir, fname)
            try:
                shutil.copyfile(src_path, dst_path)
            except:
                pass  # Ignore corrupted images

# Split cats and dogs
split_data(os.path.join(source_dir, 'glioma_tumor'),
           os.path.join(train_dir, 'glioma_tumor'),
           os.path.join(test_dir, 'glioma_tumor'))

split_data(os.path.join(source_dir, 'meningioma_tumor'),
           os.path.join(train_dir, 'meningioma_tumor'),
           os.path.join(test_dir, 'meningioma_tumor'))

split_data(os.path.join(source_dir, 'normal'),
           os.path.join(train_dir, 'normal'),
           os.path.join(test_dir, 'normal'))

split_data(os.path.join(source_dir, 'pituitary_tumor'),
           os.path.join(train_dir, 'pituitary_tumor'),
           os.path.join(test_dir, 'pituitary_tumor'))

In [ ]:
import os

# Count number of images in each set
for split in ['train', 'test']:
    print(f"\n{split.upper()} SET:")
    for category in ['glioma_tumor', 'meningioma_tumor', 'normal', 'pituitary_tumor']:
        folder_path = f"/content/output/{split}/{category}"
        count = len(os.listdir(folder_path))
        print(f"  {category}: {count} images")


In [ ]:
 from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(256, 256, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')  # For 4 classes
])

# Compile
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers, models

# model = models.Sequential([
#     layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
#     layers.MaxPooling2D(2, 2),
#         Dropout(0.25),


#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D(2, 2),
#         Dropout(0.25),


#     layers.Conv2D(128, (3, 3), activation='relu'),
#     layers.MaxPooling2D(2, 2),
#         Dropout(0.25),


#     layers.Flatten(),
#     layers.Dense(512, activation='relu'),
#         Dropout(0.5),
#     layers.Dense(4, activation='softmax')  # Change to softmax if more than two classes
# ])

# model.compile(optimizer='adam',
#               loss='categorical_crossentropy',  # Use 'categorical_crossentropy' for multi-class
#               metrics=['accuracy'])

# model.summary()

In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/content/output/train',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'  # or 'categorical' for multi-class classification
)

validation_generator = test_datagen.flow_from_directory(
    '/content/output/test',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)


In [ ]:
import math # import math module
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

history = model.fit(
    train_generator,
    steps_per_epoch=math.ceil(train_generator.samples / train_generator.batch_size), # use batch_size from train_generator
    epochs=20,
    validation_data=validation_generator,
    validation_steps=math.ceil(validation_generator.samples / validation_generator.batch_size), # use batch_size from validation_generator
    callbacks=[early_stop, reduce_lr]
)



Epoch 1/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 47s 533ms/step - accuracy: 0.2714 - loss: 4.4072 - val_accuracy: 0.2319 - val_loss: 1.9582 - learning_rate: 0.0010
Epoch 2/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 74s 477ms/step - accuracy: 0.3101 - loss: 1.8371 - val_accuracy: 0.3478 - val_loss: 1.5991 - learning_rate: 0.0010
Epoch 3/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 41s 477ms/step - accuracy: 0.2800 - loss: 1.5849 - val_accuracy: 0.2915 - val_loss: 1.5093 - learning_rate: 0.0010
Epoch 4/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 37s 479ms/step - accuracy: 0.3124 - loss: 1.4840 - val_accuracy: 0.2963 - val_loss: 1.5002 - learning_rate: 0.0010
Epoch 5/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 37s 476ms/step - accuracy: 0.3514 - loss: 1.4226 - val_accuracy: 0.3092 - val_loss: 1.4899 - learning_rate: 0.0010
Epoch 6/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 37s 477ms/step - accuracy: 0.4009 - loss: 1.3493 - val_accuracy: 0.3237 - val_loss: 1.4797 - learning_rate: 0.0010
Epoch 7/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 37s 477ms/step - accuracy: 0.4283 - loss: 1.

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'bo-', label='Training accuracy')
plt.plot(epochs, val_acc, 'ro-', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'bo-', label='Training loss')
plt.plot(epochs, val_loss, 'ro-', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()


In [ ]:
test_loss, test_acc = model.evaluate(validation_generator, steps=50)
print('Test accuracy:', test_acc)


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the image
img = image.load_img('/content/Data/pituitary_tumor/P_107.jpg', target_size=(256, 256))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Create batch dimension
img_array /= 255.0  # Normalize pixel values

# Make prediction
prediction = model.predict(img_array)

# Define the class names corresponding to the model output indices
# Adjust the list to reflect the actual class order used during training
class_names = ['glioma_tumor', 'meningioma_tumor', 'normal', 'pituitary_tumor']

# Get the index of the highest probability
predicted_index = np.argmax(prediction)
predicted_class = class_names[predicted_index]

print("Prediction:", predicted_class)
